# Análisis y Modelo de Regresión Logística — Titanic

Este notebook realiza un análisis paso a paso y entrena un **modelo de regresión logística** para predecir la supervivencia en el dataset del Titanic.

Se incluyen: descripción de valores faltantes, estrategias de imputación, codificación de variables categóricas, entrenamiento del modelo y evaluación (accuracy).

In [2]:
# Importaciones
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib
import os

print("Librerías importadas correctamente.")

Librerías importadas correctamente.


In [3]:
# Cargar datasets (asegúrate que train.csv y test.csv estén en el mismo directorio)
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

print("Train shape:", train.shape)
print("Test shape:", test.shape)

train.head()

Train shape: (891, 12)
Test shape: (418, 11)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## a) Relevamiento de valores faltantes

A continuación se muestra la cantidad de valores faltantes por variable para `train` y `test`. 

In [ ]:
# Conteo de valores faltantes
missing_train = train.isnull().sum().sort_values(ascending=False)
missing_test = test.isnull().sum().sort_values(ascending=False)

print("Valores faltantes - train:\n", missing_train[missing_train>0])
print("\nValores faltantes - test:\n", missing_test[missing_test>0])

## b) Estrategia para completar valores faltantes

- **Age**: imputaremos con la mediana (agrupando por `Sex` y `Pclass` para ser más precisos).
- **Fare**: imputación por la mediana (solo aparece 1 faltante en test).
- **Cabin**: extraeremos la letra del camarote (Deck). Para los valores faltantes usaremos un valor 'M' (Missing).
- **Embarked**: imputar con la moda (valor más frecuente).

Usaremos un `ColumnTransformer` y un `Pipeline` para encadenar imputación, codificación y escalado.

In [ ]:
# Feature engineering: extraer 'Deck' desde 'Cabin' y 'Title' desde 'Name'
def extract_deck(cabin):
    if pd.isna(cabin):
        return 'M'  # Missing
    return str(cabin)[0]

def extract_title(name):
    # extrae título desde el nombre (Mr., Mrs., Miss., etc.)
    if pd.isna(name):
        return 'Unknown'
    parts = name.split(',')
    if len(parts) > 1:
        title_section = parts[1]
        title = title_section.split('.')[0].strip()
        return title
    return 'Unknown'

train['Deck'] = train['Cabin'].apply(extract_deck)
test['Deck'] = test['Cabin'].apply(extract_deck)

train['Title'] = train['Name'].apply(extract_title)
test['Title'] = test['Name'].apply(extract_title)

# Mostrar algunas filas
train[['Name','Title','Cabin','Deck']].head()

### Preparación de variables para el modelo
Seleccionaremos estas features: `Pclass`, `Sex`, `Age`, `SibSp`, `Parch`, `Fare`, `Embarked`, `Deck`, `Title`. 

In [ ]:
# Variables a usar
features = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked','Deck','Title']
target = 'Survived'

X = train[features].copy()
y = train[target].copy()
X_test = test[features].copy()

X.shape, X_test.shape

In [ ]:
# Transformaciones por tipo de variable
numeric_features = ['Age','SibSp','Parch','Fare']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_features = ['Pclass','Sex','Embarked','Deck','Title']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

In [ ]:
# Pipeline completo con regresión logística
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

# Dividir en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Entrenar
model.fit(X_train, y_train)

# Evaluar en conjunto de validación
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f'Accuracy en validación: {accuracy:.4f}')

# Matriz de confusión y reporte
print('\nMatriz de confusión:')
print(confusion_matrix(y_val, y_pred))
print('\nReporte de clasificación:')
print(classification_report(y_val, y_pred))

In [ ]:
# Evaluación adicional con cross-validation (5 folds)
cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
print('Cross-val accuracy (5 folds):', cv_scores)
print('Media CV accuracy: {:.4f} (+/- {:.4f})'.format(cv_scores.mean(), cv_scores.std()))

#### Nota sobre la imputación de `Age`
Además de la imputación por mediana global usada en el pipeline, una alternativa es imputar `Age` usando la mediana por `Sex` y `Pclass`. A continuación se muestra cómo hacerlo y volver a entrenar si se desea.

In [ ]:
# Imputación avanzada de Age por Sex + Pclass (opcional)
X_adv = X.copy()
X_adv['Age'] = X_adv.groupby(['Sex','Pclass'])['Age'].apply(lambda grp: grp.fillna(grp.median()))

# Si aún quedan NaN (grupos sin datos), llenar con mediana global
X_adv['Age'] = X_adv['Age'].fillna(X_adv['Age'].median())

# Re-entrenar un modelo rápido para comparar
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

model_adv = Pipeline(steps=[
    ('preprocessor', preprocessor),  # preprocessor rellenará numicamente Age si queda NaN
    ('classifier', LogisticRegression(max_iter=1000))
])
scores_adv = cross_val_score(model_adv, X_adv, y, cv=5, scoring='accuracy')
print('CV accuracy con imputación por grupo (5 folds):', scores_adv)
print('Media:', scores_adv.mean(), 'Std:', scores_adv.std())

## Predicción sobre el set `test` y guardado de archivo de salida
El dataset `test.csv` no contiene la columna `Survived`. Por eso medimos accuracy usando una partición del `train`. De todas formas generaremos predicciones sobre `test` y las guardaremos en `submission.csv`. 

In [ ]:
# Reentrenar el modelo con todo el set de train
model.fit(X, y)

# Predecir sobre test
test_preds = model.predict(X_test)

submission = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': test_preds
})

output_path = '/mnt/data/submission_titanic_logreg.csv'
submission.to_csv(output_path, index=False)
print('Archivo de predicción guardado en:', output_path)

In [ ]:
# Guardar modelo entrenado para uso posterior
model_path = '/mnt/data/titanic_logreg_pipeline.joblib'
joblib.dump(model, model_path)
print('Pipeline guardado en:', model_path)

## Conclusiones

- Se completaron los valores faltantes con estrategias razonables (medianas y modas).
- Se entrenó un primer modelo de regresión logística.
- Se reportó el accuracy en un conjunto de validación y con validación cruzada.

Si querés, puedo:
- Probar otras técnicas de imputación (KNNImputer, tasa de missing indicator, etc.).
- Probar otros modelos (Random Forest, XGBoost) y comparar métricas.
- Afinar hiperparámetros con GridSearchCV o RandomizedSearchCV.
